# StateFlow - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu `StateFlow`.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExcGszZTlwNm5rMDlkdmU3d3FqY21mY2dzMno1cHM5NTM2aHVrcGd5MyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6jHVl7R4F1qkoDXK1R/giphy.gif" width="200" />

`StateFlow` rozszerza funkcjonalność strumieni `Flow` poprzez wprowadzenie pojęcia stanu (*state*). Jest to strumień, który reprezentuje zmieniający się stan i informuje obserwatorów o jego aktualnym stanie.

Główne cechy StateFlow:
- Stan: `StateFlow` przechowuje aktualny stan jako niezmienną wartość. Kiedy stan ulega zmianie, nowa wartość jest emitowana do obserwatorów.
- Emitowanie i odbieranie: `StateFlow` emituje wartości stanu i umożliwia ich odbieranie przez obserwatorów. Obserwatorzy są automatycznie informowani o aktualizacjach stanu.

`StateFlow` jest gorącym strumieniem (*hot stream*), co oznacza że emitowanie wartości odbywa się niezależnie od subskrypcji - może emitować wartości nawet wtedy, gdy nie ma aktywnych odbiorców. Nowi odbiorcy, którzy dołączają do strumienia, otrzymują **aktualną wartość stanu** oraz kolejne emitowane wartości.

Jest to zasadnicza różnica między `StateFlow` a `Flow` (cold stream). W przypadku zimnego strumienia, emisja wartości rozpoczyna się dopiero po subskrypcji przez odbiorców.

Nasza aplikacja będzie prostym licznikiem, w którym po przyciśnięciu przycisku zmienimy wartość `StateFlow` i zwiększymy stan licznika.

Rozpocznijmy od zaimplementowania `CounterViewModel`

In [ ]:
class CounterViewModel : ViewModel() {

    private val _stateFlow = MutableStateFlow(0)
    val stateFlow = _stateFlow.asStateFlow()

    fun increase(){
        _stateFlow.value += 1
    }
}

- `private val _stateFlow = MutableStateFlow(0)` - `MutableStateFlow` jest używane do przechowywania i emitowania zmieniającego się stanu - inicjujemy wartością 0
- `val stateFlow = _stateFlow.asStateFlow()` - Tworzy publiczne, tylko do odczytu `StateFlow` poprzez konwersję `MutableStateFlow` na `StateFlow` wykorzystując metofę `asStateFlow()`. `StateFlow` to niemutowalny strumień wartości, który można subskrybować i odczytywać.
- `fun increase() { _stateFlow.value += 1 }` - Zwiększa wartość `_stateFlow.value` o 1. Zmiana wartości `_stateFlow.value` spowoduje automatyczną emisję nowej wartości do subskrybentów.

Zaimplementujmy ui, oraz dodajmy fragment go jako subskrybenta `stateFlow` - implementacja jest niemal identyczna jak w przypadku `Flow`

In [ ]:
viewLifecycleOwner.lifecycleScope.launch {
    viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
        viewModel.stateFlow.collectLatest{ counter ->
            binding.counterText.text = counter.toString()
        }
    }
}

binding.increaseButton.setOnClickListener {
    viewModel.increase()
}

Różnicą jest zastosowanie funkcji `collectLatest`.  Metoda działa podobnie do `collect`, ale z jedną różnicą. Jeśli w trakcie przetwarzania emitowanych wartości pojawi się kolejna wartość, zostanie przerwane przetwarzanie bieżącej wartości i rozpocznie przetwarzanie nowej wartości. Oznacza to, że tylko ostatnia wartość emitowana w danym okresie jest przetwarzana, a poprzednie wartości są pomijane.

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExcGszZTlwNm5rMDlkdmU3d3FqY21mY2dzMno1cHM5NTM2aHVrcGd5MyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6jHVl7R4F1qkoDXK1R/giphy.gif" width="200" />